In [2]:
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split

In [3]:
data_source = '../data/raw/raw_data.csv'
data = pd.read_csv(data_source)

In [ ]:
print(data.head())

In [ ]:
print(data.info())

In [ ]:
print(data.describe())

In [ ]:
unique_categories = data['category'].unique()
unique_categories.sort()
print(unique_categories)


In [ ]:
category_counts = data['category'].value_counts()
print(category_counts)

rss_perex analysis

In [ ]:
def plot_histogram(data, column_name):
    temp_data = data.copy()
    
    column_name_length = f'{column_name}_length'
    temp_data[column_name_length] = temp_data[column_name].str.len()

    plt.figure(figsize=(10, 6))
    plt.hist(temp_data[column_name_length], bins=50, color='skyblue', edgecolor='black')
    plt.title(f'Histogram of Lengths of {column_name}')
    plt.xlabel(f'Length of {column_name}')
    plt.ylabel('Frequency')
    plt.grid(True)
    plt.show()

In [ ]:
def print_data_infos(data, column_name):
    # Calculate the lengths of titles without generating a warning
    data['length'] = data[column_name].str.len()

    # Find the shortest and longest title lengths directly
    shortest_title = data['length'].min()
    longest_title = data['length'].max()

    # Calculate the percentiles
    percentiles = data['length'].quantile([0.05, 0.5, 0.95])

    print(f"Length of shortest {column_name}: {shortest_title}")
    print(f"Length of longest {column_name}: {longest_title}")
    print("Percentiles:")
    print(percentiles)

In [ ]:
plot_histogram(data, 'rss_perex')

In [ ]:
print_data_infos(data, 'rss_perex')

In [ ]:
min_length = 30
max_length = 700
filtered_data = data[(data['length'] >= min_length) & (data['length'] <= max_length)]
print(len(filtered_data))
plot_histogram(filtered_data, 'rss_perex')

In [ ]:
plot_histogram(filtered_data, 'rss_title')

In [ ]:
print_data_infos(filtered_data, 'rss_title')

In [ ]:
min_title_length = 15
max_title_length = 100
filtered_data = filtered_data[(filtered_data['length'] >= min_title_length) & (filtered_data['length'] <= max_title_length)]
print(len(filtered_data))
plot_histogram(filtered_data, 'rss_title')

In [ ]:
# Assuming filtered_data is already defined and contains the necessary columns
filtered_data = filtered_data.drop(columns=['length'])
filtered_data['text'] = filtered_data['rss_title'] + ' ' + filtered_data['rss_perex']

# Encode the 'category' column and start the codes from 0
filtered_data['encoded_text'] = filtered_data['category'].astype('category').cat.codes

# Shift the codes by the minimum value to ensure they start from 0
filtered_data['encoded_text'] -= filtered_data['encoded_text'].min()

# Select the desired columns for the result
result = filtered_data[['text', 'encoded_text']]

In [ ]:
unique_values = result['encoded_text'].unique()
print(unique_values)

In [ ]:
# Assuming 'result' is your dataframe and 'encoded_text' is the column you are interested in
train_data, test_data = train_test_split(result, test_size=0.15, random_state=42)

# Extract unique values
train_unique_values = train_data['encoded_text'].unique()
test_unique_values = test_data['encoded_text'].unique()


import numpy as np
# Sort the unique values
sorted_train_unique_values = np.sort(train_unique_values)
sorted_test_unique_values = np.sort(test_unique_values)

print(sorted_train_unique_values)
print(sorted_test_unique_values)

In [ ]:
train_data.to_csv('../data/processed/train.csv', index=False,encoding='utf-8')
test_data.to_csv('../data/processed/test.csv', index=False,encoding='utf-8')